<a href="https://colab.research.google.com/github/reitezuz/18NES1-2025-/blob/main/week7/multiclass_classification_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass classification example - classifying digits from the MNIST dataset

Inspired by: https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter02_mathematical-building-blocks.ipynb  

MNIST dataset is a dataset of handwritten digits. It contains a training set of 60000 greyscale 28x28 images and a testing set of 10000 images of digits written by different people.

https://yann.lecun.com/exdb/mnist/

https://en.wikipedia.org/wiki/MNIST_database


For further reference datasets for deep learning, investigate:
https://keras.io/api/datasets/

## Load, observe and analyze the data

In [ ]:
# Load the MNIST dataset
import keras


# Load the MNIST dataset
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()



Observe the data:

In [ ]:
import numpy as np

# 60000 training samples - images 28x28 in greyscale
print(train_images.shape, train_labels.shape)

# 10000 testing samples - images 28x28 in greyscale
print(test_images.shape, test_labels.shape)

# 10 categories
print(len(train_labels), train_labels[:10], np.min(train_labels), np.max(train_labels))

print("Extremes of pixel values:", np.min(train_images), np.max(train_images))

# Distribution of training and testing labels
print("Label distribution:", np.bincount(train_labels))
print("Label distribution:", np.bincount(test_labels))

# Check for missing values
print("Number of missing values in training images:", np.sum(np.isnan(train_images)))
print("Number of missing values in test images:", np.sum(np.isnan(test_images)))

In [ ]:
# Display some images

import matplotlib.pyplot as plt

# Display the first 10 images
plt.figure(figsize=(10, 10))
for i in range(10):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])  # Remove axis ticks and the grids
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary) # Display the current image using a binary color map
    plt.xlabel(train_labels[i])
plt.show()


**Observation:**
1. The letters are well centered and similar in size, no values ​​are missing, there are no missing values -> we need no data augmentation
2. The input data have the form of 3D-tensor.
3. The pixels have values 0...255.

## Preprocess the data


In [ ]:
import keras
# 1. Reshape and normalize the data:
# reshape the data into a flat vector (784 elements) for input to our MLP neural network.
x_train_0 = train_images.reshape(60000, 28 * 28)
x_test_0 = test_images.reshape(10000, 28 * 28)
y_train = train_labels
y_test = test_labels

# 2. Convert the pixel values from integers [0-255] to floating-point numbers and normalize them to the range [0, 1].
x_train = x_train_0.astype('float32') / 255
x_test = x_test_0.astype('float32') / 255

# 3. Arbitrary: one-hot encode the labels:
# For example, the label 3 would become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].
y_train_categorical = keras.utils.to_categorical(y_train, num_classes=10)
y_test_categorical  = keras.utils.to_categorical(y_test, num_classes=10)

# print(x_train[1])
print(y_train[:3])
print(y_train_categorical[:3])

# 4. Split the training data into training and validation sets
# The validation set is used to monitor the performance of the model during training and prevent overfitting.
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)



## Define and train the model

### MLP model for multiclass classification:
- 'softmax' activation function in the output layer
- 'relu' or 'tanh' in the hidden layers
- if labels are one-hot vectors:
    CategoricalCrossentropy  loss function and CategoricalAccuracy metrics

- if labels are provided as integers:
     SparseCategoricalCrossentropy  loss function and SparseCategoricalAccuracy metrics


In [ ]:
# Data frame for results
import pandas as pd

columns = ["Model Name", "Test Accuracy", "Test Loss", "Train Accuracy", "Train Loss", "Time (s)", "Epochs", "Details"]
results_df = pd.DataFrame(columns=columns)


In [ ]:
# plot the training progress:
def plot_history(history):
    history_dict = history.history
    print(history_dict.keys())

    from matplotlib import pyplot as plt

    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()



In [ ]:
# Set some of the hyperparameters:
do_early_stopping = True
max_epochs = 10
batch_size = 128
num_neur_1 = 512 # number of neurons in the first hidden layer
# num_neur_2 = 50 # number of neurons in the second hidden layer
hidden_activation = 'relu'
do_tensorboard = True

###############################################
# Define the model architecture
from keras import layers
model = keras.Sequential([
    layers.InputLayer(shape=(28 * 28,)),    # Input layer
    #layers.Dense(num_neur_1, activation='relu', kernel_initializer='he_normal', bias_initializer='zeros') # First hidden layer
    layers.Dense(num_neur_1, activation=hidden_activation),
    # layers.Dense(num_neur_2, activation=hidden_activation),
    layers.Dense(10, activation='softmax')  # Output layer for multiclass classification
])
import datetime
model_name = "mnist_mlp_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.keras'
model.summary()

# Configure the model:
model.compile(optimizer=keras.optimizers.SGD(learning_rate = 0.001), # Adam, RMSProp
              loss= keras.losses.SparseCategoricalCrossentropy(),
              metrics= [keras.metrics.SparseCategoricalAccuracy("accuracy")])

###############################################
# Define callbacks (e.g., early stopping):
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
callbacks = [early_stopping] if do_early_stopping else []
if do_tensorboard:
    from keras.callbacks import TensorBoard
    tensorboard_callback = TensorBoard(log_dir="./logs_mnist/"+model_name, histogram_freq=1, write_steps_per_second=True)
    callbacks.append(tensorboard_callback)

################################################
# Train the model
import time
start_time = time.time()
history = model.fit(x_train,
                    y_train,
                    epochs=max_epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    callbacks=[tensorboard_callback])
time_fit = time.time() - start_time

###############################
# Plot the training progress:
plot_history(history)

# Evaluate the model on the training, validation and test sets
train_loss, train_acc = model.evaluate(x_train, y_train)
val_loss, val_acc = model.evaluate(x_val, y_val)
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Training accuracy:', train_acc, '\nTrain loss:', train_loss)
print('Validation accuracy:', val_acc, '\nVal loss:', val_loss)
print('Test accuracy:', test_acc, '\nTest loss:', test_loss)

###############################
# Save the model:
import os
model_dir = "./models/"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model.save(model_dir + model_name)

#################################
# Add results to the dataframe:
model_details = f"{num_neur_1}-{hidden_activation}-{do_early_stopping}-ep.:{max_epochs}-bs:{batch_size}"
new_entry = {
    "Model Name" : model_name,
    "Details" : model_details,
    "Test Accuracy" : test_acc,
    "Test Loss" : test_loss,
    "Train Accuracy" : train_acc,
    "Train Loss" : train_loss,
    "Time (s)" : time_fit,
    "Epochs" : len(history.epoch),
}
# View and and save the dataframe:
results_df.to_csv(model_dir + "mnist_results.csv", index=False)
print("Results:")
print(results_df)

In [ ]:
# plot the training progress:
import matplotlib.pyplot as plt

history_dict = history.history
print(history_dict.keys())

# Plot training & validation accuracy values
plt.plot(history_dict['accuracy'])
plt.plot(history_dict['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_dict['loss'])
plt.plot(history_dict['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()



## Evaluate the model and make predictions on new data

In [ ]:
# Get predicted probabilities for the test set
y_pred_probs = model.predict(x_test)
print(y_pred_probs[0])

# Get the predicted class for each sample
y_pred = np.argmax(y_pred_probs, axis=1)

print("Predicted labels:", y_pred[:10])
print("True labels:", y_test[:10])

# Misclassified indices:
misclassified_indices = np.where(y_pred != y_test)[0]
num_misclassified = len(misclassified_indices)
print("Number of misclassified images:", num_misclassified,
      "out of", len(y_test), ", accuracy", test_acc)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Plot some misclassified images
num_images_to_plot = 5
plt.figure(figsize=(10, 10))
for i in range(min(num_images_to_plot, len(misclassified_indices))):
    index = misclassified_indices[i]
    plt.subplot(1, num_images_to_plot, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[index], cmap=plt.cm.binary)
    plt.xlabel(f"Pred: {y_pred[index]}, True: {y_test[index]}")
plt.show()

In [ ]:
# Plot some misclassified images from a given target (or predicted) class
target_class = 3
misclassified_indices_class = np.where((y_pred != y_test) & (y_test == target_class))[0]
#misclassified_indices_class = np.where((y_pred != y_test) & (y_pred == target_class))[0]



# Display the first 25 misclassified images for the target class
plt.figure(figsize=(10, 10))
for i in range(min(25, len(misclassified_indices_class))):
    index = misclassified_indices_class[i]
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[index], cmap=plt.cm.binary)
    plt.xlabel(f"True:{y_test[index]}, Pred:{y_pred[index]}")
plt.show()


In [ ]:
###############################################
# Load TensorBoard notebook extension
%load_ext tensorboard

# Start TensorBoard before training begins
%tensorboard --logdir logs/fit_mnist --reload_interval=1

# Exercises
1. **Change the number of layers and neurons in the model**. Observe how this affects the model's accuracy. You can also experiment with different **activation functions** in the hidden layers.
2. **Experiment with the number of epochs and learning rate**. Plot the learning curves to visualize the differences in training.
3. **Change the loss function**. Try using `categorical_crossentropy` with one-hot encoded labels (`y`).
4. **Experiment with different batch sizes**. Observe how different batch sizes affect the model's accuracy and training time.
5. **Try an alternative normalization method for the input data** (e.g., standardization). Observe how this affects the model's performance.
6. **Analyze the confusion matrix**. Use the confusion matrix to identify pairs of classes that are most frequently misclassified.
7. **Try changing the optimizer used in the model**. Experiment with optimizers such as Adam, RMSprop, or Adagrad to see how they impact model accuracy and training time.
